In [2]:
# Importing required modules
import pandas as pd          
import numpy as np               # For mathematical calculations 
import matplotlib.pyplot as plt  # For plotting graphs 
import datetime as dt
from datetime import datetime    # To access datetime 
from pandas import Series        # To work on series 
%matplotlib inline 
from ucimlrepo import fetch_ucirepo
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import sktime
from sktime.datasets import load_from_tsfile_to_dataframe

# Settings for pretty nice plots
plt.style.use('fivethirtyeight')
plt.show()

## Get Dataset

In [44]:
# fetch dataset 
air_quality = fetch_ucirepo(id=360) 
  
# data (as pandas dataframes) 
X = air_quality.data.features 

y = X['CO(GT)']

print(X.shape)

X = X.drop('CO(GT)', axis=1)
  
# metadata 
# print(air_quality.metadata) 
  
# variable information 
# print(air_quality.variables) 

print(X.shape)
print(y.shape)

(9357, 15)
(9357, 14)
(9357,)


In [32]:
# Preprocess the data
X = X.fillna(0)  # Fill missing values with 0
y = y.astype(int)  # Convert targets to integers

X = X.apply(pd.to_numeric, errors='coerce')

In [33]:
class AirQualityDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.long)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create data loaders
batch_size = 32
train_dataset = AirQualityDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

### Create RNN Model

In [38]:
class AirQualityRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, dropout):
        super().__init__()
        
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers=n_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        output, (hidden, cell) = self.rnn(x)
        return self.fc(hidden[-1])

In [45]:
input_dim = X.shape[1]
hidden_dim = 64
output_dim = 1
n_layers = 2
dropout = 0.5

# Instantiate the model
model = AirQualityRNN(input_dim, hidden_dim, output_dim, n_layers, dropout)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

RuntimeError: size mismatch (got input: [1], target: [32])